In [85]:
import re
import pandas as pd
import numpy as np

# Función para procesar una línea de texto y extraer la información
def procesar_linea(linea, serie_actual):
    numero = ''
    titulo = ''
    
    # Buscar coincidencias con el patrón de números y HS (para número de tomos)
    match = re.match(r'^\s*(\d+|HS\d*|SN)\.?\s*(.*)', linea)
    
    if match:
        numero = match.group(1).strip()
        titulo = match.group(2).strip()
        return serie_actual, numero, titulo
    return None

# Leer el archivo de texto
def procesar_archivo_txt(nombre_archivo):
    datos = []
    serie_actual = None
    
    with open(nombre_archivo, 'r', encoding='utf-8') as archivo:
        for linea in archivo:
            linea = linea.strip()
            
            # Si encontramos el nombre de la serie
            if re.match(r'^[A-ZÀ-ÖØ-Ýa-zà-öø-ý][A-ZÀ-ÖØ-Ýa-zà-öø-ý ]+\(.+\)$', linea):  # Maneja nombres con acentos/tildes
                serie_actual = re.split(r'\s*\(.+\)', linea)[0].strip()
            else:
                procesado = procesar_linea(linea, serie_actual)
                if procesado:
                    datos.append(procesado)

    return datos

archivo_txt = '/users/el_fer/Repos/my_ds_notebooks/100 applied/010 Bibliotheque Primaire Saint Michel/inventaire_BD_sept_2024.txt'
archivo_csv_salida = 'libros.csv'  # Nombre del archivo CSV de salida

datos = procesar_archivo_txt(archivo_txt)
df = pd.DataFrame(datos, columns=['Serie', 'Numero', 'Titre'])

In [86]:
df

,Serie,Numero,Titre
0,ANATOLE LATUILE,1,C'est parti !
1,ANATOLE LATUILE,2,Oohiohioo !
2,ANATOLE LATUILE,3,Personne en vue !
3,ANATOLE LATUILE,4,Record battu !
4,ANATOLE LATUILE,5,Ultra top secret !
...,...,...,...
1044,ZARLA,4,Rage
1045,ZARLA,5,Les lueurs vénéneuses
1046,ZARLA,1,Tome 1
1047,ZARLA,2,Tome 2


In [87]:
df['Unites']=1

In [88]:
df['Unites'] = np.where(df['Titre'].str.contains('2x'), 2, df['Unites'])
df['Unites'] = np.where(df['Titre'].str.contains('3x'), 3, df['Unites'])
df['Unites'] = np.where(df['Titre'].str.contains('4x'), 4, df['Unites'])

In [89]:
df['Commentaire'] = np.where(df['Titre'].str.contains('(abimé)'), '(1 abîmé)', '')
df['Commentaire'] = np.where(df['Titre'].str.contains('(abîmé)'), '(1 abîmé)', '')
df['Commentaire'] = np.where(df['Titre'].str.contains('1 abimé'), '1 abîmé', '')

In [90]:
df['Commentaire'] = np.where(df['Titre'].str.contains('1 abîmé'), '1 abîmé', df['Commentaire'])

In [91]:
df[df['Titre'].str.contains('1 ab')]

,Serie,Numero,Titre,Unites,Commentaire
273,GAME OVER,2,No problemo (2x) (1 abîmé),2,1 abîmé
276,GAME OVER,5,Walking Blork (2x) (1 abîmé),2,1 abîmé
406,LES LÉGENDAIRES,3,Le Cimetière des autobus (3x) (1 abîmé),3,1 abîmé
407,LES LÉGENDAIRES,4,Idylles (2x) (1 abîmé),2,1 abîmé
775,LES SISTERS,1,Un air de famille (3x) (1 abimé),3,1 abîmé
777,LES SISTERS,3,C’est elle qui a commencé (2x) (1 abimé),2,1 abîmé
779,LES SISTERS,5,Quelle chouchoute! (2x) (1 abîmé),2,1 abîmé


In [92]:
df['Titre'] = df['Titre'].str.replace('(2x)', '')

In [93]:
def clean_titre(titre):
    # Remove occurrences of (2x), (3x), (1 abimé) using regex
    titre_cleaned = re.sub(r'\(\d+x\)|\(1 abimé\)|\(1 abîmé\)', '', titre)
    # Replace double spaces with single spaces
    titre_cleaned = re.sub(r'\s+', ' ', titre_cleaned)
    # Strip leading and trailing spaces
    return titre_cleaned.strip()

# Apply the cleaning function to the 'Titre' column
df['Titre'] = df['Titre'].apply(clean_titre)

In [94]:
df.head(20)

,Serie,Numero,Titre,Unites,Commentaire
0,ANATOLE LATUILE,1,C'est parti !,1,
1,ANATOLE LATUILE,2,Oohiohioo !,1,
2,ANATOLE LATUILE,3,Personne en vue !,1,
3,ANATOLE LATUILE,4,Record battu !,1,
4,ANATOLE LATUILE,5,Ultra top secret !,1,
5,ANATOLE LATUILE,6,Pas de panique !,1,
6,ANATOLE LATUILE,7,Ça va dégominer !,1,
7,ANATOLE LATUILE,8,Le roi du chahut,1,
8,ANATOLE LATUILE,9,Ça déménage !,1,
9,ANATOLE LATUILE,10,Sauve qui peut,1,


In [95]:
df.to_csv(archivo_csv_salida, index=False, encoding='utf-8')